These first two notebook code blocks were ran through google colab after uploading my cleaned dataset to google drive, with purpose to split data into train, validate, test. The rest of the code was ran locally through VS code, and then uploaded.

In [ ]:
import os
import pandas as pd
from git import Repo
from pathlib import Path
import ast
from tqdm import tqdm

# === PATH SETUP ===
save_dir = "/Users/paulmitchell/Downloads/Gen_AI_Homework_2"
csv_path = os.path.join(save_dir, "results.csv")
clone_dir = os.path.join(save_dir, "cloned_repos")
processed_log = os.path.join(save_dir, "processed_repos.txt")
output_csv = os.path.join(save_dir, "raw_methods_dataset.csv")
index_file = os.path.join(save_dir, "last_repo_index.txt")
os.makedirs(clone_dir, exist_ok=True)

# === LOAD REPOS FROM CSV ===
df = pd.read_csv(csv_path)
repo_names = df["name"].dropna().tolist()
repo_urls = [f"https://github.com/{name}" for name in repo_names]
print(f"📦 Loaded {len(repo_urls)} total repositories to process.")

📦 Loaded 7677 total repositories to process.


In [ ]:
index_file = "/Users/paulmitchell/Downloads/Gen_AI_Homework_2/last_repo_index.txt"
repo_index = 4268
method_count = 108324

with open(index_file, "w") as f:
    f.write(f"{repo_index}\n{method_count}")

print("✅ Updated resume checkpoint.")

✅ Updated resume checkpoint.


In [ ]:
import os
import pandas as pd
import ast
import subprocess
from pathlib import Path
from tqdm import tqdm

# === PARAMETERS ===
max_total = 120_000
max_per_file = 5
max_ifs_per_repo = 100

index_file = os.path.join(save_dir, "last_repo_index.txt")
output_csv = os.path.join(save_dir, "raw_methods_dataset.csv")
processed_log = os.path.join(save_dir, "processed_repos.txt")
csv_path = output_csv

# === AUTO-VALIDATE resume state ===
if os.path.exists(csv_path) and os.path.exists(index_file):
    df = pd.read_csv(csv_path)
    with open(index_file, "r") as f:
        lines = f.read().splitlines()
        saved_count = int(lines[1]) if len(lines) > 1 else 0
        actual_count = len(df)
        if actual_count != saved_count:
            print(f"⚠️ Mismatch: index file says {saved_count}, but CSV has {actual_count} rows.")
        else:
            print(f"✅ Verified: {actual_count} methods match the index file.")

# === INITIAL VALUES ===
start_index = 0
current_total = 0

# === LOAD DATASET IF EXISTS ===
if os.path.exists(output_csv):
    print("📂 Resuming from existing dataset...")
    df_existing = pd.read_csv(output_csv)
    dataset = df_existing.to_dict(orient="records")
    print(f"🔁 Loaded {len(dataset)} existing samples.")
else:
    dataset = []

# === LOAD PROCESSED REPOS ===
if os.path.exists(processed_log):
    with open(processed_log, "r") as f:
        processed = set(f.read().splitlines())
else:
    processed = set()

# === LOAD INDEX AND METHOD COUNT IF EXISTS ===
if os.path.exists(index_file):
    with open(index_file, "r") as f:
        lines = f.read().splitlines()
        if len(lines) >= 1:
            start_index = int(lines[0])
        if len(lines) >= 2:
            current_total = int(lines[1])
    print(f"⏩ Resuming from repo #{start_index + 1} with {current_total} methods")

# === SAVE INDEX HELPER ===
def save_progress(index_file_path, repo_index, method_count):
    with open(index_file_path, "w") as f:
        f.write(f"{repo_index}\n{method_count}")

# === CLONE REPO WITH TIMEOUT ===
def clone_repo(repo_url, dest_dir=clone_dir, timeout=60):
    repo_name = repo_url.strip("/").split("/")[-1]
    target_path = os.path.join(dest_dir, repo_name)
    if os.path.exists(target_path):
        return target_path
    try:
        subprocess.run(
            ["git", "clone", repo_url + ".git", target_path],
            check=True,
            timeout=timeout,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
    except subprocess.TimeoutExpired:
        print(f"⏱️ Timed out cloning {repo_url}")
        return None
    except subprocess.CalledProcessError:
        print(f"❌ Failed to clone {repo_url}")
        return None
    return target_path

# === EXTRACT METHODS WITH `if` ===
def extract_if_methods_from_file(file_path, max_per_file=5):
    try:
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            source = f.read()
        tree = ast.parse(source)
    except Exception:
        return []

    methods = []

    class Visitor(ast.NodeVisitor):
        def visit_FunctionDef(self, node):
            if any(isinstance(n, ast.If) for n in ast.walk(node)):
                methods.append(node)
    Visitor().visit(tree)

    return methods[:max_per_file]

# === MASK FIRST `if` LINE ===
def mask_first_if(node: ast.FunctionDef):
    try:
        source = ast.unparse(node)
        lines = source.splitlines()
        for i, line in enumerate(lines):
            stripped = line.strip()
            if stripped.startswith("if ") and stripped.endswith(":"):
                condition = stripped[3:-1].strip()
                lines[i] = "<mask>:"
                return "\n".join(lines), condition
    except Exception:
        pass
    return None, None

# === MAIN LOOP ===
repo_bar = tqdm(repo_urls[start_index:], initial=start_index, total=len(repo_urls), dynamic_ncols=True)

for repo_index, repo_url in enumerate(repo_urls[start_index:], start=start_index):
    if repo_url in processed:
        continue

    repo_bar.set_description(f"🧪 Repo {repo_index + 1}/{len(repo_urls)} | {current_total}/120k methods")

    repo_path = clone_repo(repo_url)
    if not repo_path:
        continue

    py_files = list(Path(repo_path).rglob("*.py"))
    if not py_files:
        continue

    repo_ifs_collected = 0
    for file_path in py_files:
        methods = extract_if_methods_from_file(file_path, max_per_file=max_per_file)
        for method in methods:
            if repo_ifs_collected >= max_ifs_per_repo:
                break

            masked_func, target_block = mask_first_if(method)
            if masked_func and target_block:
                token_count = len(masked_func.split())
                dataset.append({
                    "cleaned_method": masked_func,
                    "target_block": target_block,
                    "tokens_in_method": token_count
                })
                current_total += 1
                repo_ifs_collected += 1

                if current_total >= max_total:
                    break
        if current_total >= max_total or repo_ifs_collected >= max_ifs_per_repo:
            break

    if repo_ifs_collected == 0:
        print(f"⚠️ No if-methods found in {repo_url}")

    # ✅ Log repo and progress
    with open(processed_log, "a") as f:
        f.write(repo_url + "\n")

    save_progress(index_file, repo_index + 1, current_total)
    pd.DataFrame(dataset).to_csv(output_csv, index=False)

    if current_total >= max_total:
        break

# === FINAL SAVE ===
pd.DataFrame(dataset).to_csv(output_csv, index=False)
print(f"\n✅ Done! Saved {len(dataset)} samples to {output_csv}")


✅ Verified: 16895 methods match the index file.
📂 Resuming from existing dataset...
🔁 Loaded 16895 existing samples.
⏩ Resuming from repo #693 with 16895 methods


🧪 Repo 694/7677 | 16895/120k methods:   9%|▉         | 692/7677 [13:17:44<?, ?it/s]
🧪 Repo 695/7677 | 16895/120k methods:   9%|▉         | 692/7677 [01:00<?, ?it/s]

⏱️ Timed out cloning https://github.com/macarthur-lab/clinvar


🧪 Repo 716/7677 | 17542/120k methods:   9%|▉         | 692/7677 [03:16<?, ?it/s]

⏱️ Timed out cloning https://github.com/ghadjeres/deepbach


...

⏱️ Timed out cloning https://github.com/hgjazhgj/fgo-py


🧪 Repo 4262/7677 | 108108/120k methods:   9%|▉         | 692/7677 [7:26:33<?, ?it/s]

⏱️ Timed out cloning https://github.com/typenetwork/roboto-flex


🧪 Repo 4268/7677 | 108324/120k methods:   9%|▉         | 692/7677 [7:27:18<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import pandas as pd

input_path = "/Users/paulmitchell/Downloads/Gen AI 520/Hw2/raw_methods_dataset.csv"
output_path = "/Users/paulmitchell/Downloads/Gen AI 520/Hw2/cleaned_dataset.csv"

df = pd.read_csv(input_path)
print(f"Raw rows: {len(df)}")

# Drop empty rows
df = df.dropna(subset=["cleaned_method", "target_block"])

# Filter by token length
df = df[(df["tokens_in_method"] >= 10) & (df["tokens_in_method"] <= 300)]

# Remove duplicates
df = df.drop_duplicates(subset=["cleaned_method"])

print(f"Cleaned rows: {len(df)}")
df.to_csv(output_path, index=False)
print(f"Cleaned dataset saved to: {output_path}")


📦 Raw rows: 108324
🧼 Cleaned rows: 98614
✅ Cleaned dataset saved to: /Users/paulmitchell/Downloads/Gen AI 520/Hw2/cleaned_dataset.csv


In [ ]:
import pandas as pd

# Change path to either cleaned or sampled file
df = pd.read_csv("/Users/paulmitchell/Downloads/Gen AI 520/Hw2/cleaned_dataset.csv")

# View 3 random samples
for i, row in df.sample(3, random_state=None).iterrows():
    print(f"\n🔍 Sample #{i}")
    print("Cleaned Method:\n", row["cleaned_method"])
    print("Target Block:", row["target_block"])
    print("Token Count:", row["tokens_in_method"])


🔍 Sample #74249
Cleaned Method:
 def report(self):
    results_by_status = collections.defaultdict(list)
    for result in self.results:
<mask>:
            results_by_status['successful'].append(result)
        elif result['failure'] or result['error']:
            results_by_status['failed'].append(result)
        elif result['interrupted']:
            results_by_status['interrupted'].append(result)
        else:
            results_by_status['unknown'].append(result)
    if self.options.summary_mode:
        self.report_failures(results_by_status['failed'])
    self.report_stats(len(self.test_case_classes), **results_by_status)
    if len(self.results) == 0:
        return False
    else:
        return len(results_by_status['failed']) + len(results_by_status['interrupted']) + len(results_by_status['unknown']) == 0
Target Block: result['success']
Token Count: 41

🔍 Sample #75137
Cleaned Method:
 def history_paths() -> list:
    """
    Get valid pathes to history from HISTORIES va

In [ ]:
import pandas as pd

# Path to cleaned dataset in Drive
full_path = '/content/drive/MyDrive/Gen AI Homework 2/dataset/cleaned_dataset.csv'

# Load the full cleaned dataset
df = pd.read_csv(full_path)
print(f"Loaded dataset with {len(df)} rows")

# Take first 60k to match assignment's 50k train + 5k val + 5k test
df = df.iloc[:60000].reset_index(drop=True)

# Split datasets
train_df = df.iloc[:50000]
val_df = df.iloc[50000:55000]
test_df = df.iloc[55000:60000]

# Save splits to new CSVs
save_dir = '/content/drive/MyDrive/Gen AI Homework 2/dataset'
train_df.to_csv(f'{save_dir}/train.csv', index=False)
val_df.to_csv(f'{save_dir}/val.csv', index=False)
test_df.to_csv(f'{save_dir}/test.csv', index=False)

print("Saved:")
print("train.csv:", len(train_df))
print("val.csv:", len(val_df))
print("test.csv:", len(test_df))